# Pagerank on subgraphs—efficient Monte-Carlo estimation

In this repo you can find the reference code for my novel Subrank algorithm for efficiently computing the Pagerank distribution over $S$ subgraph of $G$.
For the reasoning behind the algorithm, the definition and the analysis, I invite the interested reader to [read the paper](https://pippellia.com/pippellia/Social+Graph/Pagerank+on+subgraphs%E2%80%94efficient+Monte-Carlo+estimation).

To play with it, follow these steps:

## Step 0: execute the necessary functions

In [4]:
import json
import networkx as nx

def load_network(name):

    '''
    This function loads the network graph and index map from storage.

    INPUTS
    ------
    name: str
        the name of the files, usually a timestamp like '1714823396'

    OUTPUTS:
    -------
    index_map: dict
        The dictionary {pk: node} that maps each public key to its corrispondent node id in the graph

    network_graph: graph
        A Networkx graph.
    '''

    if type(name) != str:
        name = str(name)

    # loading the index_map
    with open('index_map_' + name + '.json', 'r') as f:
        index_map = json.load(f)
    
    # loading the JSON for the graph
    with open('network_graph_' + name + '.json', 'r') as f:
        data = json.load(f)

    # convert JSON back to graph
    network_graph = nx.node_link_graph(data)

    return index_map, network_graph


import numpy as np
import random
from scipy.sparse import lil_matrix, csr_matrix

def get_mc_pagerank(G, R, nodelist = None, alpha=0.85):
    
    '''
        Monte-Carlo complete path stopping at dandling nodes
    
        INPUTS
        ------
        G: graph
            A directed Networkx graph. This function cannot work on directed graphs.
            
        R: int
            The number of random walks to be performed per node
            
        nodelist: list, optional
            the list of nodes in G networkx graph. 
            It is used to order the nodes in a specified way
            
        alpha: float, optional
            It is the dampening factor of Pagerank. default value is 0.85

        OUTPUTS
        -------
        walk_visited_count: CSR matrix
            a Compressed Sparse Row (CSR) matrix; element (i,j) is equal to 
            the number of times v_j has been visited by a random walk started from v_i
            
        mc_pagerank: dict
            The dictionary {node: pg} of the pagerank value for each node in G

        References
        ----------
        [1] K.Avrachenkov, N. Litvak, D. Nemirovsky, N. Osipova
        "Monte Carlo methods in PageRank computation: When one iteration is sufficient"
        https://www-sop.inria.fr/members/Konstantin.Avratchenkov/pubs/mc.pdf
    '''

    # validate all the inputs and initialize variables
    N, nodelist, inverse_nodelist = _validate_inputs_and_init_mc(G, R, nodelist, alpha)

    # initialize walk_visited_count as a sparse LIL matrix
    walk_visited_count = lil_matrix((N, N), dtype='int')

    progress_count = 0

    # perform R random walks for each node
    for node in nodelist:

        # print progress every 200 nodes
        progress_count += 1
        if progress_count % 200 == 0:
            print('progress = {:.2f}%'.format(100 * progress_count / N), end='\r')
        
        for _ in range(R):

            node_pos = inverse_nodelist[node]
            walk_visited_count[node_pos, node_pos] += 1

            current_node = node

            while random.uniform(0,1) < alpha:
                
                successors = list(G.successors(current_node))
                if not successors:
                    break
                    
                current_node = random.choice(successors)
                current_node_pos = inverse_nodelist[current_node]

                # add current node to the walk_visited_count
                walk_visited_count[node_pos, current_node_pos] += 1

    # convert lil_matrix to csr_matrix for efficient storage and access
    walk_visited_count = walk_visited_count.tocsr()

    # sum all visits for each node into a numpy array
    total_visited_count = np.array(walk_visited_count.sum(axis=0)).flatten()

    # reciprocal of the number of total visits
    one_over_s = 1 / sum(total_visited_count)
    
    mc_pagerank = {nodelist[j]: total_visited_count[j] * one_over_s for j in range(N)}

    print('progress = 100%       ', end='\r')
    print('\nTotal walks performed: ', N * R )
    
    return walk_visited_count, mc_pagerank


def _validate_inputs_and_init_mc(G, R, nodelist, alpha):

    '''
    This function validate the inputs and initialize the following variables:
    
    N: int
        the number of nodes in G Networkx graph

    nodelist : list
        the list of nodes in G Networkx graph

    inverse_nodelist : dict
       a dictionary that maps each node in G to its position in nodelist
    '''

    N = len(G)
    if N == 0:
        raise ValueError("Graph G is empty")
    
    if not isinstance(R, int) or R <= 0:
        raise ValueError("R must be a positive integer")
    
    if not isinstance(alpha, float) or not (0 < alpha < 1):
        raise ValueError("alpha must be a float between 0 and 1")
    
    if nodelist is not None and set(nodelist) != set(G.nodes()):
        raise ValueError("nodelist does not match the nodes in G")

    elif nodelist is None:
        nodelist = list(G.nodes())

    # compute the inverse map of nodelist
    inverse_nodelist = {nodelist[j]: j for j in range(N)}

    return N, nodelist, inverse_nodelist


import networkx as nx
import numpy as np
from scipy.sparse import isspmatrix_csr
import random

def get_subrank(S, G, walk_visited_count, nodelist, alpha = 0.85):

    '''
        Subrank algorithm (stopping at dandling nodes);
        it aims to approximate the Pagerank over S subgraph of G
    
        INPUTS
        ------
        S: graph
            A directed Networkx graph, induced subgraph of G
            
        G: graph
            A directed Networkx graph. This function cannot work on directed graphs.
            
        walk_visited_count: CSR matrix
            a Compressed Sparse Row (CSR) matrix; element (i,j) is equal to 
            the number of times v_j has been visited by a random walk started from v_i

        nodelist: list, optional
            the list of nodes in G Networkx graph. It is used to decode walk_visited_count
        
       alpha: float, optional
            It is the dampening factor of Pagerank. default value is 0.85

        OUTPUTS
        -------
        subrank: dict
            The dictionary {node: pg} of the pagerank value for each node in S

        References
        ----------
        [1] Pippellia,
        "Pagerank on subgraphs—efficient Monte-Carlo estimation"
        https://pippellia.com/pippellia/Social+Graph/Pagerank+on+subgraphs%E2%80%94efficient+Monte-Carlo+estimation
    '''
    
    # validate inputs and initialize variables
    N, S_nodes, G_nodes, inverse_nodelist = _validate_inputs_and_init(S, G, walk_visited_count, nodelist, alpha)

    # compute visited count from walks that started from S
    visited_count_from_S = _get_visited_count_from_S(N, S_nodes, walk_visited_count, nodelist, inverse_nodelist)

    # compute positive and negative walks to do
    positive_walks, negative_walks = _get_walks_to_do(S_nodes, G_nodes, S, G, visited_count_from_S, alpha)

    print(f'walks performed = {sum(positive_walks.values()) + sum(negative_walks.values())}')

    # perform the walks and get the visited counts
    positive_count = _perform_walks(S_nodes, S, positive_walks, alpha)
    negative_count = _perform_walks(S_nodes, S, negative_walks, alpha)

    # add the effects of the random walk to the count of G
    new_visited_count = {node: visited_count_from_S[node] + positive_count[node] - negative_count[node]
                        for node in S_nodes}

    # compute the number of total visits
    total_visits = sum(new_visited_count.values())

    # compute the subrank
    subrank = {node: visits / total_visits 
                   for node, visits in new_visited_count.items() }

    return subrank

def _validate_inputs_and_init(S, G, walk_visited_count, nodelist, alpha):

    '''
    This function validate the inputs and initialize the following variables:
    
    N: int
        the number of nodes in G Networkx graph

    S_nodes: set
        the set of nodes that belongs to S

    G_nodes: set
        the set of nodes that belongs to G

    inverse_nodelist : dict
        a dictionary that maps each node in G to its position in nodelist
        
    Note: S being a subgraph of G is NOT checked because it's computationally expensive.
    '''

    if len(S) == 0:
        raise ValueError("graph S is empty")

    N = len(G)
    if N == 0:
        raise ValueError("graph G is empty")
    
    if not isinstance(alpha, float) or not (0 < alpha < 1):
        raise ValueError("alpha must be a float between 0 and 1")

    if not isspmatrix_csr(walk_visited_count) or walk_visited_count.shape != (N,N):
        raise ValueError(f"walk_visited_count must be a {(N,N)} CSR matrix")

    S_nodes = set(S.nodes())
    G_nodes = set(G.nodes())
    
    if not nodelist or set(nodelist) != set(G_nodes):
        raise ValueError("nodelist does not match the nodes in G")

    # compute the inverse map of nodelist
    inverse_nodelist = {nodelist[j]: j for j in range(N)}

    return N, S_nodes, G_nodes, inverse_nodelist


def _get_visited_count_from_S(N, S_nodes, walk_visited_count, nodelist, inverse_nodelist):

    '''
    This function extracts the number of visits that come from walks that started from S
    '''

    # getting the indices of nodes in S
    S_indices = [inverse_nodelist[node] for node in S_nodes]
    
    # Extract the rows
    S_matrix = walk_visited_count[S_indices, :]

    # Sum the rows
    visited_count_from_S = np.array(S_matrix.sum(axis=0)).flatten()

    # convert to a dictionary
    visited_count_from_S = {nodelist[j]: visited_count_from_S[j] for j in range(N)}
    
    return visited_count_from_S
    

def _get_walks_to_do(S_nodes, G_nodes, S, G, visited_count_from_S, alpha):

    '''
    This function calculates the positive and negative walks to be done for each node.
    It is a necessary step to take into account the different structure of S
    with respect to that of G.
    '''

    # compute nodes in G-S
    external_nodes = G_nodes - S_nodes

    # compute nodes in S that point to G-S
    nodes_that_point_externally = {u for u,v in nx.edge_boundary(G, S_nodes, external_nodes)}

    walks_to_do = {node: 0 for node in S_nodes}

    # add positive random walks to walks_to_do
    for node in nodes_that_point_externally:

        successors = set(G.successors(node)) & S_nodes

        if successors:

            # compute estimate visits
            visited_count = visited_count_from_S[node]
            degree_S = S.out_degree(node)
            degree_G = G.out_degree(node)
            estimate_visits = alpha * visited_count * (1/degree_S - 1/degree_G)

            for succ in successors:
                walks_to_do[succ] += estimate_visits

    # subtract number of negative random walks
    for node in external_nodes:

        successors = set(G.successors(node)) & S_nodes

        if successors:
            
            # compute estimate visits
            visited_count = visited_count_from_S[node]
            degree = G.out_degree(node)
            estimate_visits = alpha * visited_count / degree

            for succ in successors:
                walks_to_do[succ] -= estimate_visits

    # split the walks to do into positive and negative
    positive_walks_to_do = {node: round(value) for node,value in walks_to_do.items() if value > 0 }
    negative_walks_to_do = {node: round(-value) for node,value in walks_to_do.items() if value < 0 }

    return positive_walks_to_do, negative_walks_to_do
    

def _perform_walks(S_nodes, S, walks_to_do, alpha):

    '''
    This function performs a certain number of random walks on S for each node;
    It then returns the visited count for each node in S.
    '''

    # initializing the visited count
    visited_count = {node: 0 for node in S_nodes}

    for starting_node in walks_to_do.keys():

        num = walks_to_do[starting_node]

        # performing num random walks
        for _ in range(num):

            current_node = starting_node
            visited_count[current_node] += 1

            # performing one random walk
            while random.uniform(0,1) < alpha:
    
                successors = list(S.successors(current_node))
        
                if not successors:
                    break
    
                current_node = random.choice(successors)

                # updating the visited count
                visited_count[current_node] += 1

    return visited_count

## Step 1: load the graph database

First, you have to load the networkx graph database into memory by running the following code.

In [1]:
import time

# loading the database
print('loading the database...')
tic = time.time()

index_map, G = load_network(1714823396)

toc = time.time()
print(f'finished in {toc-tic} seconds')

loading the database...
finished in 17.221821308135986 seconds


## Step 2: Compute Pagerank over $G$

Compute the pagerank over $G$ by using the networkx built-in pagerank function that uses the power iteration method.
This vector will be considered as the real Pagerank vector and will be used to compute the errors of the Monte-Carlo algorithm.

In [2]:
# computing the pagerank
print('computing global pagerank...')
tic = time.time()

p_G = nx.pagerank(G, tol=1e-12)

toc = time.time()
print(f'finished in {toc-tic} seconds')

computing global pagerank...
finished in 18.110231637954712 seconds


## Step 3: Approximate Pagerank over $G$ using Monte-Carlo

Compute the pagerank over $G$ using a simple Monte-Carlo implementation and compute the L1 error.
This step is essential because it returns the csr-matrix `walk_visited_count`, that will be used later by the Subrank algorithm.

In [5]:
# number of the random walks per node
R = 10

# fix the order of the nodes
nodelist = list(G.nodes())

tic = time.time()

# perform the random walks and get the monte-carlo pagerank
walk_visited_count, mc_pagerank = get_mc_pagerank(G, R, nodelist)

toc = time.time()
print(f'performed random walks in {toc-tic} seconds')

# computing the L1 error
error_G_mc = sum( abs(p_G[node] - mc_pagerank[node])
                  for node in G.nodes() )

print(f'error pagerank vs mc pagerank in G = {error_G_mc}')

progress = 100%       
Total walks performed:  1951740
performed random walks in 35.56719255447388 seconds
error pagerank vs mc pagerank in G = 0.05047368343396303


## Step 4: Select random subgraph $S$ and compute its Pagerank distribution

Select a random subgraph $S$ consisting of 50k nodes, and compute its Pagerank distribution.

In [6]:
# selecting random subgraph S
S_nodes = set(random.sample(list(G.nodes()), k=50000))
S = G.subgraph(S_nodes).copy()

# computing pagerank over S
print('computing local pagerank...')
tic = time.time()

p_S = nx.pagerank(S, tol=1e-12)

toc = time.time()
print(f'finished in {toc-tic} seconds')

computing local pagerank...
finished in 2.2579879760742188 seconds


## Step 5: Approximate Pagerank over $S$ using Subrank

Run the Subrank algorithm to approximate the Pagerank over $S$ subgraph of $G$. Then compute the L1 error.

In [7]:
# computing subrank
print('computing subrank over S...')
tic = time.time()

subrank = get_subrank(S, G, walk_visited_count, nodelist)

toc = time.time()
print(f'performed random walks in {toc-tic} seconds')

# computing the L1 error
error_S_subrank = sum( abs(p_S[node] - subrank[node])
                      for node in S_nodes )

print(f'error pagerank vs subrank in S = {error_S_subrank}')

computing subrank over S...
walks performed = 162010
performed random walks in 2.6661059856414795 seconds
error pagerank vs subrank in S = 0.05139446533060389


## Step 6: Approximate Pagerank over $S$ using Monte-Carlo naive recomputation

Run the Monte-Carlo Pagerank algorithm on $S$ as a reference for the number of random walks required and the error achieved.

In [8]:
# computing the monte-carlo pagerank 
print('computing naive monte-carlo pagerank over S')
tic = time.time()

_, mc_pagerank_S_naive = get_mc_pagerank(S,R)

toc = time.time()
print(f'finished in {toc-tic} seconds')

# computing the L1 error
error_S_naive = sum( abs(p_S[node] - mc_pagerank_S_naive[node])
                      for node in S.nodes())

print(f'error pagerank vs mc pagerank in S = {error_S_naive}')

computing naive monte-carlo pagerank over S
progress = 100%       
Total walks performed:  500000
finished in 4.987701177597046 seconds
error pagerank vs mc pagerank in S = 0.04875892632550595
